In [1]:
%matplotlib inline
from IPython.display import clear_output

In [2]:
import numpy as np
import pylab as plt

from xgboost import XGBClassifier
from utils.data import load_new_data
from utils.pass_quality import approx_pq
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, InputLayer, GRU, SimpleRNN

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 4007)


ImportError: cannot import name InputLayer

In [204]:
def nn_model(X_train):
    model = Sequential()
    model.add(InputLayer(input_shape=[X_train.shape[1]]))
    model.add(Dense(12, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',
                  loss='MSE',
                  metrics=['accuracy'])
    return model

def rnn_model(X_train):
    

# default classifiers like XGBoost, Logistic Regression

In [269]:
errors = []

for iteration in range(10):
    data = load_new_data()
    X, y = data[['r.shield', 'r.loop', 'r.shield']].values, data['y'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    xgb = XGBClassifier()
    #lr = LogisticRegression()
    #nn = nn_model(X_train)

    xgb.fit(X_train, y_train)
    #lr.fit(X_train, y_train)
    #nn.fit(X_train, y_train, nb_epoch=4, verbose=False)

    y_pred_xgb = xgb.predict(X_test)
    #y_pred_lr = lr.predict(X_test)
    #y_pred_nn = (nn.predict(X_test) > 0.5).astype(int).flatten()
    errors.append(approx_pq(y_test, y_pred_xgb)[:-1])

In [279]:
np.matrix(errors).T[1].mean()

2621.40000001

# data augmentation

In [80]:
def dim_extend(X, y, n_prev = 3):
    """"""
    X_ext, y_ext = [], []
    
    for i in range(len(X) - n_prev + 1):
        X_ext.append(np.concatenate([X[i+j] for j in range(n_prev)]))
        y_ext.append(y[i + n_prev - 1])
        #docY_lab.append(y_lab[i + n_prev -1])
    X_ext = np.array(X_ext)
    y_ext = np.array(y_ext)
    #alsY_lab = np.array(docY_lab)
    return X_ext, y_ext#, alsY_lab

# xgb model

In [281]:
errors_xgb = []

for n_prev in range(3, 25):
    print "computing for ", n_prev
    clear_output(wait=True)
    X_e, y_e = dim_extend(X, y, n_prev)
    X_train_e, X_test_e = X_e[:int(len(X_e) * 0.75)], X_e[int(len(X_e) * 0.75):]
    y_train_e, y_test_e = y_e[:int(len(y_e) * 0.75)], y_e[int(len(y_e) * 0.75):]
    xgb = XGBClassifier()
    xgb.fit(X_train_e, y_train_e)
    y_pred_e = xgb.predict(X_test_e)
    errors_xgb.append(approx_pq(y_test_e, y_pred_e)[:-1])

computing for  24


In [296]:
print (np.mean(np.matrix(errors_xgb).T[0]) + np.max(np.matrix(errors_xgb).T[0])) / 2

print (np.mean(np.matrix(errors_xgb).T[1]) + np.max(np.matrix(errors_xgb).T[1])) / 2
#print np.max(np.matrix(errors_nn).T[1])

print (np.mean(np.matrix(errors_xgb).T[2]) + np.max(np.matrix(errors_xgb).T[2])) / 2
#print np.max(np.matrix(errors_nn).T[2])

0.769984322484
1740.75000001
552.136363636


# logreg model

In [282]:
errors_lr = []

for n_prev in range(3, 30):
    print "computing for ", n_prev
    clear_output(wait=True)
    X_e, y_e = dim_extend(X, y, n_prev)
    X_train_e, X_test_e = X_e[:int(len(X_e) * 0.75)], X_e[int(len(X_e) * 0.75):]
    y_train_e, y_test_e = y_e[:int(len(y_e) * 0.75)], y_e[int(len(y_e) * 0.75):]
    lr = LogisticRegression()
    lr.fit(X_train_e, y_train_e)
    y_pred_e = lr.predict(X_test_e)
    errors_lr.append(approx_pq(y_test_e, y_pred_e)[:-1])

computing for  29


In [295]:
print (np.mean(np.matrix(errors_lr).T[0]) + np.max(np.matrix(errors_lr).T[0])) / 2

print (np.mean(np.matrix(errors_lr).T[1]) + np.max(np.matrix(errors_lr).T[1])) / 2
#print np.max(np.matrix(errors_nn).T[1])

print (np.mean(np.matrix(errors_lr).T[2]) + np.max(np.matrix(errors_lr).T[2])) / 2
#print np.max(np.matrix(errors_nn).T[2])

0.791391927479
1809.0925926
649.37037037


# nn model:

In [283]:
errors_nn = []

for n_prev in range(20, 25):
    print "computing for ", n_prev
    clear_output(wait=True)
    X_e, y_e = dim_extend(X, y, n_prev)
    X_train_e, X_test_e = X_e[:int(len(X_e) * 0.75)], X_e[int(len(X_e) * 0.75):]
    y_train_e, y_test_e = y_e[:int(len(y_e) * 0.75)], y_e[int(len(y_e) * 0.75):]
    nn = nn_model(X_train_e)
    nn.fit(X_train_e, y_train_e, nb_epoch=int(n_prev / 6.) + 6, verbose=False)
    y_pred_e = nn.predict(X_test_e)
    errors_nn.append(approx_pq(y_test_e, (nn.predict(X_test_e) > 0.5).astype(int).flatten())[:-1])

computing for  24


In [294]:
print (np.mean(np.matrix(errors_nn).T[0]) + np.max(np.matrix(errors_nn).T[0])) / 2

print (np.mean(np.matrix(errors_nn).T[1]) + np.max(np.matrix(errors_nn).T[1])) / 2
#print np.max(np.matrix(errors_nn).T[1])

print (np.mean(np.matrix(errors_nn).T[2]) + np.max(np.matrix(errors_nn).T[2])) / 2
#print np.max(np.matrix(errors_nn).T[2])

0.771217306473
1742.80000001
546.2


# benchmark:

In [310]:
errors = []

for iteration in range(10):
    data = load_new_data()
    y_lab, y = data[['r.pass']].values, data['y'].values
    _, y = y[:int(len(y_e) * 0.75)], y[int(len(y_e) * 0.75):]
    _, y_lab = y_lab[:int(len(y_e) * 0.75)], y_lab[int(len(y_e) * 0.75):]

    #xgb = XGBClassifier()
    #lr = LogisticRegression()
    #nn = nn_model(X_train)

    #xgb.fit(X_train, y_train)
    #lr.fit(X_train, y_train)
    #nn.fit(X_train, y_train, nb_epoch=4, verbose=False)

    #y_pred_xgb = xgb.predict(X_test)
    #y_pred_lr = lr.predict(X_test)
    #y_pred_nn = (nn.predict(X_test) > 0.5).astype(int).flatten()
    errors.append(approx_pq(y_lab.flatten(), y)[:-1])

In [313]:
print np.mean(np.matrix(errors).T[0])

print np.mean(np.matrix(errors).T[1])
#print np.max(np.matrix(errors_nn).T[1])

print np.mean(np.matrix(errors).T[2])
#print np.max(np.matrix(errors_nn).T[2])

0.842790279319
1730.10000001
322.9


# RNN model